# Django Administrator

**Note: The entire project folder from session 13 was copied to session 14 since we'll need most of the stuff we already build.**

Django is a marvel when it comes to database and administration, from it's Object Relational Mapping system to the fact that it has a built in Admin page! If you navigate to the `domain.name/admin` URL, you'll see a login page designed for django administration. But what is the username as password? And what are the things the admin can do? Those are the topics we'll talk about in this session.

First things first, Admin is a User in the database which is exactly like any other user in terms of username, passowrd, email, and all the attributes of the User table in the database. Except, it has access to many things we'll talk about later on. Let's start by creating an admin using Django CLI.

In [ ]:
python manage.py createsuperuser

After running this command you'll be prompted to type username, email address and password. Type those and then take a look at database. You'll find the user you just created. However, if you take a look at the password column, you'll see something like this:

```raw
pbkdf2_sha256$870000$tBnP35MQfPQUI7DzkGJW9k$daDy/GtxgC3nGVe289Ayon5JVdDp5um17b3OGioWi1E=
```

This is a greate opportunity to step aside from the main topic just for a moment and talk about how Django deals with sensitive information like passwords.

The string above, includes three parts: Hash algorithm (`pbkdf2_sha256`), Salt (`tBnP35MQfPQUI7DzkGJW9k`) and the rest is a password hash. Hash funcitons are algorithms that can encrypt any string into a fixed sized string (88 characters in this case) which cannot be decrypted back to the original string. Changing a single character in the original password will result in an entirely different hash. 

Note that Hash functions cannot be reverse engineered, keeping the passwords safe even if the datbase was compromised. The only way to decrypt a hashed string is to brute-force all possible strings to see which one would result the same hash. For the most powerfull computers these days, this process might take centuries to finish. (Not for quantum computers though!)

Salt, is a random string added to users password before hashing to make the result even more random. Our friend ChatGPT here explains how this process helps the website's security.

---

#### 🔹 What Happens if the Database is Leaked?
If an attacker gets access to the database, they can see:
1. The **hashing algorithm** (e.g., `pbkdf2_sha256`).
2. The **salt** used for each user.
3. The **hashed password**.

Since the salt is **stored with the hash**, the attacker **doesn’t need to guess it**—they can use it directly. However, **salting still provides protection** because it prevents attackers from using **precomputed hashes (rainbow tables)**.

#### 🔹 Can an Attacker Crack the Passwords?
Yes, but it's not **instant** because:
- Django uses **key-stretching algorithms** like **PBKDF2, bcrypt, or Argon2**, which are **intentionally slow** to make brute-force attacks costly.
- Each user has a **unique salt**, so the attacker **cannot** precompute a list of hashes (**rainbow table attack is useless**).
- They have to brute-force each password **one by one**, which takes significantly longer.

#### 🔹 How an Attacker Might Proceed:
1. They take a common password (e.g., `"123456"`).
2. They hash it using the stolen salt.
3. If the generated hash matches a stored hash, they have cracked that password.
4. They repeat this for millions of passwords (**brute force** or a **dictionary attack**).

Since Django uses **many iterations** (default: **600,000 for PBKDF2**), each attempt is slow, making mass attacks expensive.

#### 🔹 How to Make Passwords Even More Secure:
1. **Use a strong hashing algorithm**  
   Django’s default (`PBKDF2`) is good, but `Argon2` is even better for new projects:
   ```python
   PASSWORD_HASHERS = [
       'django.contrib.auth.hashers.Argon2PasswordHasher',
   ]
   ```
2. **Increase iterations** for PBKDF2:
   ```python
   PASSWORD_HASHERS = [
       ('django.contrib.auth.hashers.PBKDF2PasswordHasher', {'iterations': 1200000}),
   ]
   ```
3. **Use two-factor authentication (2FA)**  
   Even if an attacker cracks a password, they won’t get in without 2FA.
4. **Monitor for breaches**  
   Services like **Have I Been Pwned** can help detect if users’ passwords have been leaked.

---

A lot of these items are recomended automatically by Django. You can apply new password validators or alter old ones in project settings at `AUTH_PASSWORD_VALIDATORS`. More on that later, enough with the nerd talk, let's learn how admins work

### Admin Panel

Navigate back to the URL pointing to admin panel: `localhost/admin` and fill in the username and password you created for the superuser in the Shell earlier.

The first thing you see is a list of available data for the admin to alter. Initially you'd see `Users` and `Groups` in that list. Feel free to play around and even break some things. Just kidding, try not to break anything, but head up for CyberSec enthusiasts: Django Adming Panel is one of the cool vulnerabilities to research on. 

One very cool detail is when you change the app language in project settings to farsi, the panel changes to Farsi too! Meaning that Django in fact has a support for Farsi.

Note that the product database we created earlier does not exist here, in order to be able to perform CRUD on that table as well, the `Product` model needs to be registered to the admin. To do so, navigate to `admin.py` file in the app directory and update it as follows:

In [ ]:
from django.contrib import admin
from . import models

# Register your models here.

admin.site.register(models.Product)

Reload the website and you should be good to go.

As a programmer, we have the highest level of access to the website and management systems, even higher that the admin. As the good old uncle Ben said: 

We are the ones, in charge of developing a robust system that even admins won't be able to break so easily. One of the most crucial attributes of our `Product` model here is the `slug`, the variable in charge of creating the SEO and User-Friendly URL for each instance. Each slug has to be unique and not blank, but admins might accidentally delete or alter slugs in a way that would break the product. Basically, slug is something that is better the admins had no control over. To implement this limitaion, we'll use our knowledge from Database Design and Data Strucures as follows:

In [ ]:
from django.db import models
from django.utils.text import slugify
import string, random

class Product(models.Model):
    name = models.CharField(default="N/A", max_length=100, blank=False)
    price = models.IntegerField(default=0)
    is_active = models.BooleanField(default=True)
    slug = models.SlugField(default="", unique=True, blank=True, db_index=True)

    def __str__(self):
        return f"{self.name}"
    
    # Override the save method
    def save(self, *args, **kwargs):
        # Generate slug directly from product name
        base_slug = slugify(self.name)
        chars = string.ascii_letters
        guess = base_slug

        # Store the similar slugs in a set to look up
        existing_slugs = set(
            Product.objects.filter(slug__startswith=base_slug)
            .values_list("slug", flat=True)
        )

        while guess in existing_slugs:
            salt = ''.join(random.choices(chars, k=3))
            guess = base_slug + salt
        
        self.slug = guess

        # Don't forget to run the original at the end
        super().save(*args, **kwargs)

First things first, `save()` is a method called automatically anytime some product is added to database or is altered in the database. This could be done by end user, admin or even developer. This code makes sure that anytime the `save()` is performed, the altered / new product will have a slug generated from it's name; if it's not unique, a random two charachter string is added to the end of slug.

Where we used Database Design Knowledge?
The choice of using `filter()` instead of `get()`, since the database is indexed, filtering is way faster

Where we used Data Structures Knowledge?
The conversion of Django QuerySet into Python Sets using `value_list()` will result in faster checking in the while loop condition since Python Sets are HashSets behind the scenes. The search operation is extremely fast (approximately O(1))